In [1]:
import re
import time
import pandas as pd
import xarray as xr
from numpy import ones
from numpy.linalg import cholesky
from pandas_plink import read_plink1_bin
from limix.qc import quantile_gaussianize

In [2]:
import cellregmap 
cellregmap 

<module 'cellregmap' from '/hps/nobackup2/stegle/users/acuomo/git_repos/CellRegMap/cellregmap/__init__.py'>

In [3]:
from cellregmap import run_interaction

In [4]:
revision_folder = "/hps/nobackup2/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/REVISION/"

In [5]:
# run 10 tests per job

In [6]:
input_files_dir = "/hps/nobackup2/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/new/input_files/"

In [7]:
## sample mapping file
## this file will map cells to donors 
## it will also only include donors we have single-cell data for (a subset of all of HipSci donors)
sample_mapping_file = input_files_dir+"sample_mapping_file.csv"
sample_mapping = pd.read_csv(sample_mapping_file, dtype={"genotype_individual_id": str, "phenotype_sample_id": str})

In [8]:
## genotype_individual_id are donor IDs, as found in the genotype matrix (G) and GRM covariance (K)
## phenotype_sample_id are cell IDs, as found in the scRNA-seq phenotype vector (y) and cell context covariance (C)
sample_mapping.head()

,genotype_individual_id,phenotype_sample_id
0,HPSI0114i-joxm_1,21843_1#10
1,HPSI0314i-fafq_1,21843_1#100
2,HPSI0314i-fafq_1,21843_1#101
3,HPSI1013i-wuye_2,21843_1#102
4,HPSI0114i-joxm_1,21843_1#103


In [9]:
## extract unique individuals
donors = sample_mapping["genotype_individual_id"].unique()
donors.sort()
print("Number of unique donors: {}".format(len(donors)))

Number of unique donors: 126


In [10]:
############################################
################ Kinship matrix ############
############################################

In [11]:
## read in GRM (genotype relationship matrix; kinship matrix)
kinship_folder="/hps/nobackup2/stegle/users/acuomo/hipsci_genotype_files/"
kinship_file=kinship_folder+"hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.kinship"
K = pd.read_csv(kinship_file, sep="\t", index_col=0)
assert all(K.columns == K.index) #symmetric matrix, donors x donors

In [12]:
K = xr.DataArray(K.values, dims=["sample_0", "sample_1"], coords={"sample_0": K.columns, "sample_1": K.index})
K = K.sortby("sample_0").sortby("sample_1")
donors = sorted(set(list(K.sample_0.values)).intersection(donors))
print("Number of donors after kinship intersection: {}".format(len(donors)))

Number of donors after kinship intersection: 125


In [13]:
## subset to relevant donors
K = K.sel(sample_0=donors, sample_1=donors)
assert all(K.sample_0 == donors)
assert all(K.sample_1 == donors)

In [14]:
## and decompose such as K = hK @ hK.T (using Cholesky decomposition)
hK = cholesky(K.values)
hK = xr.DataArray(hK, dims=["sample", "col"], coords={"sample": K.sample_0.values})
assert all(hK.sample.values == K.sample_0.values)

In [15]:
del K
print("Sample mapping number of rows BEFORE intersection: {}".format(sample_mapping.shape[0]))
## subsample sample mapping file to donors in the kinship matrix
sample_mapping = sample_mapping[sample_mapping["genotype_individual_id"].isin(donors)]
print("Sample mapping number of rows AFTER intersection: {}".format(sample_mapping.shape[0]))

Sample mapping number of rows BEFORE intersection: 34256
Sample mapping number of rows AFTER intersection: 33964


In [16]:
############################################
##### expand from donors to cells ##########

In [17]:
## use sel from xarray to expand hK (using the sample mapping file)
hK_expanded = hK.sel(sample=sample_mapping["genotype_individual_id"].values)
assert all(hK_expanded.sample.values == sample_mapping["genotype_individual_id"].values)

In [18]:
######################################
############### Genotypes ############
######################################

In [19]:
## read in genotype file (plink format)
plink_folder = "/hps/nobackup2/stegle/users/acuomo/hipsci_genotype_files/"
plink_file = plink_folder+"hipsci.wec.gtarray.HumanCoreExome.imputed_phased.20170327.genotypes.norm.renamed.bed"
G = read_plink1_bin(plink_file)

Mapping files: 100%|██████████| 3/3 [05:50<00:00, 116.89s/it]


In [20]:
#############################
###### SNP selection

In [21]:
# filter file (columns: snp_id, gene)
fvf_filename = revision_folder+"/CRM_interaction_chr21/fvf.csv"
fvf = pd.read_csv(fvf_filename, index_col = 0)
fvf.head(20)

,feature,snpID,chrom
1,ENSG00000256073_C21orf119,21_33666631_T_C,21
2,ENSG00000256073_C21orf119,21_33667566_G_A,21
3,ENSG00000256073_C21orf119,21_33667866_C_T,21
4,ENSG00000256073_C21orf119,21_33668024_T_C,21
5,ENSG00000256073_C21orf119,21_33668131_A_G,21
6,ENSG00000256073_C21orf119,21_33668493_A_C,21
7,ENSG00000256073_C21orf119,21_33668891_A_G,21
8,ENSG00000256073_C21orf119,21_33669542_A_G,21
9,ENSG00000256073_C21orf119,21_33670219_C_T,21
10,ENSG00000256073_C21orf119,21_33670344_C_T,21


In [110]:
i = 86
genes = fvf['feature'].unique()

In [111]:
gene = genes[i]

In [112]:
fvf_gene = fvf[fvf['feature']==gene]

In [113]:
n = fvf_gene.shape[0]
n

763

In [114]:
# for i in range(0,n,10):
#     print(i)

In [ ]:
j = 60

In [ ]:
fvf_sel = fvf_gene.iloc[j:(j+10)]
fvf_sel

In [ ]:
genes = fvf_sel['feature'].unique()

In [ ]:
genes

In [ ]:
# (1) gene name (feature_id)
gene_name = genes[0]
trait_name = re.sub("_.*","",gene_name)
trait_name

In [ ]:
leads = fvf_sel[fvf_sel['feature']==gene_name]['snpID'].unique()

In [ ]:
G_sel = G[:,G['snp'].isin(leads)]

In [ ]:
G_sel.shape

In [ ]:
############################################
##### expand from donors to cells ##########

In [ ]:
# expand out genotypes from cells to donors (and select relevant donors in the same step)
G_expanded = G_sel.sel(sample=sample_mapping["genotype_individual_id"].values)
assert all(hK_expanded.sample.values == G_expanded.sample.values)

In [ ]:
G_expanded.shape

In [ ]:
######################################
############## Phenotypes ############
######################################

In [ ]:
# Phenotype (single-cell expression)
phenotype_file = input_files_dir+"phenotype.csv.pkl"
phenotype = pd.read_pickle(phenotype_file)
print("Phenotype shape BEFORE selection: {}".format(phenotype.shape))
phenotype = xr.DataArray(phenotype.values, dims=["trait", "cell"], coords={"trait": phenotype.index.values, "cell": phenotype.columns.values})
phenotype = phenotype.sel(cell=sample_mapping["phenotype_sample_id"].values)
print("Phenotype shape AFTER selection: {}".format(phenotype.shape))
assert all(phenotype.cell.values == sample_mapping["phenotype_sample_id"].values)

In [ ]:
# select gene
y = phenotype.sel(trait=gene_name)
# quantile normalise
y = quantile_gaussianize(y)

In [ ]:
######################################
########## Cell contexts #############
######################################

In [ ]:
# cells by MOFA factors (20)
C_file = "/hps/nobackup2/stegle/users/acuomo/all_scripts/struct_LMM2/sc_endodiff/debug_May2021/mofa_logcounts_model_factors.csv"
C = pd.read_csv(C_file, index_col = 0)
C = xr.DataArray(C.values, dims=["cell", "pc"], coords={"cell": C.index.values, "pc": C.columns.values})
C = C.sel(cell=sample_mapping["phenotype_sample_id"].values)
assert all(C.cell.values == sample_mapping["phenotype_sample_id"].values)

In [ ]:
# quantile normalise cell contexts
C_gauss = quantile_gaussianize(C)

In [ ]:
######################################
############ Covariates ##############
######################################

In [ ]:
n_cells = phenotype.shape[1]
W = ones((n_cells, 1))

In [ ]:
y = y.values.reshape(y.shape[0],1)
y.shape

In [ ]:
W

In [ ]:
C.values[:,0:10].shape
C.values[:,0:10]

In [ ]:
start_time = time.time()
GG = G_expanded.values
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:

GG.shape

In [ ]:
GG

In [ ]:
start_time = time.time()
pvals = run_interaction(y, W=W, E=C_gauss.values[:,0:10], E1=C_gauss.values[:,0:10], E2=C.values[:,0:20], G=GG, hK=hK_expanded)[0]
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
pv = pd.DataFrame({"chrom":G_expanded.chrom.values,
               "pv":pvals,
               "variant":G_expanded.snp.values})
pv

In [ ]:
folder = revision_folder+"CRM_interaction_chr22/results/"
outfilename = f"{folder}{trait_name}_{j}.tsv"

In [ ]:
outfilename

In [ ]:
import os
if os.path.exists(outfilename):
    print("File already exists, exiting")

In [ ]:
pv.to_csv(outfilename)